In [1]:
import numpy as np
from numba import jit

In [2]:
@jit(nopython=True)
def idx2site(Lx,Ly,idx):
    return np.vstack((idx%Lx,idx//Lx))

In [3]:
@jit(nopython=True)
def site2idx(Lx,Ly,x,y):
    return x+Lx*y

In [4]:
# @jit(nopython=True)
# def main_1():
#     Lx = 4
#     Ly = 3
#     site = np.array([1])
#     print("site --> idx",site,idx2site(Lx,Ly,site))
#     idxs = np.arange(Lx*Ly)
#     print("inds",idxs)
#     sites = idx2site(Lx,Ly,idxs)
#     print("sites",sites)
#     print("xs",sites[0])
#     print("ys",sites[1])
#     idxs2 = site2idx(Lx,Ly,sites[0],sites[1])
#     print("idxs2",idxs2)

In [5]:
# main_1()

In [6]:
@jit(nopython=True)
def makeHam(L,Lhbond,i1s,i2s,Js):
    Ham = np.zeros((L,L),dtype=np.float64)
#    Ham = np.zeros((L,L),dtype=np.complex128)
    for i in range(Lhbond):
        Ham[i1s[i],i2s[i]] = Js[i]
        Ham[i2s[i],i1s[i]] = Js[i]
    return Ham

In [7]:
@jit(nopython=True)
def main_2():
    L = 10
## periodic
#    i1s = np.arange(L)
#    i2s = (i1s + 1)%L
## open
    i1s = np.array([i for i in range(L-1)])
    i2s = np.array([(i+1)%L for i in range(L-1)])
    Js = - np.ones(L-1)
    Lhbond = len(Js)
    print("i1s",i1s)
    print("i2s",i2s)
    print("Js",Js)
    Ham = makeHam(L,Lhbond,i1s,i2s,Js)
    print(Ham)
    ene, vec = np.linalg.eigh(Ham)
    for i in range(L):
        vec[:,i] *= np.sign(vec[0,i])
    print(vec[:,0])
    print(vec[:,0].T@Ham@vec[:,0],ene[0])
#
    eps = np.array([-2.0 * np.cos((n+1) * np.pi / (L+1)) for n in range(L)])
    x = np.zeros((L,L), dtype = np.float64)
#    x = np.zeros((L,L), dtype = np.complex128)
    for j in range(L):
        for k in range(L):
            x[j,k] = np.sin((j+1)*(k+1)*np.pi / (L+1))
    x *= np.sqrt(2.0 / (L+1))
    x = x.T
    print(x[:,0])
    print(x[:,0].T@Ham@x[:,0],eps[0])
## vec=x or not
    print("sum(|vec-x|)",np.sum(np.abs(vec-x)))
## vec.T=vec or not --> yes
    print("sum(|vec.T-vec|)",np.sum(np.abs(vec.T-vec)))

In [8]:
main_2()

i1s [0 1 2 3 4 5 6 7 8]
i2s [1 2 3 4 5 6 7 8 9]
Js [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
[[ 0. -1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [-1.  0. -1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. -1.  0. -1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. -1.  0. -1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. -1.  0. -1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0. -1.  0. -1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. -1.  0. -1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -1.  0. -1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0. -1.  0. -1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0. -1.  0.]]
[0.12013117 0.23053002 0.3222527  0.38786839 0.42206128 0.42206128
 0.38786839 0.3222527  0.23053002 0.12013117]
-1.9189859472289952 -1.9189859472289947
[0.12013117 0.23053002 0.3222527  0.38786839 0.42206128 0.42206128
 0.38786839 0.3222527  0.23053002 0.12013117]
-1.918985947228995 -1.9189859472289947
sum(|vec-x|) 2.67841304690819e-14
sum(|vec.T-vec|) 2.9032332093947844e-14


In [9]:
@jit(nopython=True)
def main_3():
    Lx = 2
    Ly = 2
## open
    i1s = []
    i2s = []
    Js = []
    for y in range(Ly):
        for x in range(Lx):
            i0 = site2idx(Lx,Ly,x,y)
            if x+1<Lx:
                ix = site2idx(Lx,Ly,x+1,y)
                i1s.append(i0)
                i2s.append(ix)
                Js.append(-1.0)
            if y+1<Ly:
                iy = site2idx(Lx,Ly,x,y+1)
                i1s.append(i0)
                i2s.append(iy)
                Js.append(-1.0)
    Lhbond = len(Js)
    print("i1s",i1s)
    print("i2s",i2s)
    print("Js",Js)
    print("Lhbond",Lhbond,2*Lx*Ly-Lx-Ly)
#
## CDW init
    init_confs = np.zeros(Lx*Ly,dtype=np.int64)
    for y in range(Ly):
        for x in range(Lx):
            i0 = site2idx(Lx,Ly,x,y)
            init_confs[i0] = (1-(-1)**(x+y))//2
    print("init_confs",init_confs)
    num = np.sum(init_confs)
    print("num",num)
    init_is = np.zeros(num,dtype=np.int64)
    cnt = 0
    for i in range(Lx*Ly):
        if init_confs[i] == 1:
            init_is[cnt] = i
            cnt +=1
    print("init_is",init_is)
#
    L = Lx*Ly
    Ham = makeHam(L,Lhbond,i1s,i2s,Js)
    print(Ham)
    ene, vec = np.linalg.eigh(Ham)
    for i in range(L):
        vec[:,i] *= np.sign(vec[0,i])
    print(vec[:,0])
    print(vec[:,0].T@Ham@vec[:,0],ene[0])
    print(vec[:,1])
    print(vec[:,1].T@Ham@vec[:,1],ene[1])
#
#    eps = np.array([-2.0 * (np.cos((i+1)*np.pi/(Lx+1)) + np.cos((j+1)*np.pi/(Ly+1))) for i in range(Lx) for j in range(Ly)])
#    print(eps)
# vec.T=vec or not --> no
    print("sum(|vec.T-vec|)",np.sum(np.abs(vec.T-vec)))
#
    tstep = 0.1
    L = Lx*Ly
    L_A = L//2
#    y = vec.T @ (np.diag(np.exp(1j * ene * tstep))) @ vec
    y = (vec.T).astype(np.complex128) @ (np.diag(np.exp(1j * ene * tstep))) @ vec.astype(np.complex128)
    print("y",y)
    z = np.zeros((num,num), dtype = np.complex128)
    for j in range(num):
        ij = init_is[j]
        for k in range(num):
            ik = init_is[k]
            for m in range(L_A):
                z[j,k] += y[ij,m].conjugate() * y[ik,m]
    print("z",z)
    dlt = np.eye(num,dtype=np.complex128)
    print("dlt",dlt)
    a = np.vstack( (np.hstack((z,dlt-z)), np.hstack((dlt-z,z))) )
    print("a",a)

In [10]:
main_3()

i1s [0, 0, 1, 2]
i2s [1, 2, 3, 3]
Js [-1.0, -1.0, -1.0, -1.0]
Lhbond 4 4
init_confs [0 1 1 0]
num 2
init_is [1 2]
[[ 0. -1. -1.  0.]
 [-1.  0.  0. -1.]
 [-1.  0.  0. -1.]
 [ 0. -1. -1.  0.]]
[0.5 0.5 0.5 0.5]
-1.9999999999999991 -2.0
[ 0.70506666 -0.05367498  0.05367498 -0.70506666]
1.9721522630525295e-31 -3.3335489201032854e-16
sum(|vec.T-vec|) 3.9083500937683073
y [[ 9.90033289e-01+6.93889390e-18j  1.11022302e-16-1.40075122e-01j
  -1.11022302e-16-1.06635721e-02j -9.96671108e-03-1.38777878e-17j]
 [ 1.11022302e-16-1.40075122e-01j  9.80181434e-01+9.71445147e-17j
  -1.50873832e-03+1.04083409e-17j -1.11022302e-16-1.40075122e-01j]
 [-1.11022302e-16-1.06635721e-02j -1.50873832e-03+9.54097912e-18j
   9.99885143e-01-1.01915004e-17j  2.77555756e-16-1.06635721e-02j]
 [-9.96671108e-03-1.38777878e-17j -1.11022302e-16-1.40075122e-01j
   2.77555756e-16-1.06635721e-02j  9.90033289e-01-6.24500451e-17j]]
z [[9.80376684e-01+0.00000000e+00j 1.48638705e-05-7.23690061e-18j]
 [1.48638705e-05+7.23690061e-18